In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [8]:
def compute_fluence(E_erg, D_Mpc):
    """
    Compute fluence in J/m² for a given energy (erg) and distance (Mpc).
    """
    E_joule = E_erg * 1e-7
    D_meter = D_Mpc * 3.086e22
    return E_joule / (4 * np.pi * D_meter**2)


In [9]:
def required_energy_for_projection(D_Mpc, threshold=1e-5):
    """
    Compute the minimum energy (erg) required to exceed collapse threshold fluence at given distance.
    """
    D_meter = D_Mpc * 3.086e22
    E_joule = 4 * np.pi * D_meter**2 * threshold
    return E_joule * 1e7  # convert back to erg


In [10]:
historical_events = pd.DataFrame([
    {
        "name": "SN1987A", "type": "SN", "distance_Mpc": 0.05, "energy_erg": 1e53,
        "observed_neutrinos": True, "neutrino_detectors": ["Kamiokande-II", "IMB", "Baksan"]
    },
    {
        "name": "GW170817", "type": "NS merger", "distance_Mpc": 40, "energy_erg": 1e52,
        "observed_neutrinos": False, "neutrino_detectors": ["IceCube", "ANTARES", "Super-K"]
    },
    {
        "name": "GW150914", "type": "BH merger", "distance_Mpc": 410, "energy_erg": 5e54,
        "observed_neutrinos": False, "neutrino_detectors": ["IceCube", "ANTARES"]
    },
    {
        "name": "SN2016aps", "type": "SN", "distance_Mpc": 350, "energy_erg": 4e53,
        "observed_neutrinos": False, "neutrino_detectors": ["IceCube", "Super-K"]
    },
    {
        "name": "GW190521", "type": "BH merger", "distance_Mpc": 530, "energy_erg": 8e54,
        "observed_neutrinos": False, "neutrino_detectors": ["IceCube"]
    }
])


In [11]:
threshold = 1e-5  # Bruno collapse threshold in J/m²

def classify_event(row):
    fluence = compute_fluence(row["energy_erg"], row["distance_Mpc"])
    required = required_energy_for_projection(row["distance_Mpc"], threshold)
    return pd.Series({
        "fluence_J_per_m2": fluence,
        "log10_fluence": np.log10(fluence),
        "required_energy_erg": required,
        "log10_required_energy": np.log10(required),
        "collapse_predicted": fluence >= threshold,
        "matches_observation": (fluence >= threshold) == row["observed_neutrinos"]
    })

# Apply the classification
audit = historical_events.join(historical_events.apply(classify_event, axis=1))


In [12]:
display_cols = ["name", "type", "distance_Mpc", "energy_erg",
                "fluence_J_per_m2", "collapse_predicted", "observed_neutrinos", "matches_observation"]

print("📜 Historical Entropy Collapse Audit:")
display(audit[display_cols])


📜 Historical Entropy Collapse Audit:


,name,type,distance_Mpc,energy_erg,fluence_J_per_m2,collapse_predicted,observed_neutrinos,matches_observation
0,SN1987A,SN,0.05,1.000000e+53,334.239893,True,True,True
1,GW170817,NS merger,40.00,1.000000e+52,0.000052,True,False,False
2,GW150914,BH merger,410.00,5.000000e+54,0.000249,True,False,False
3,SN2016aps,SN,350.00,4.000000e+53,0.000027,True,False,False
4,GW190521,BH merger,530.00,8.000000e+54,0.000238,True,False,False
